<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/CNN/Deep_CIFAR_10_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [29]:
EPOCHS = 20
NUM_CLASSES = 10
INPUT_SHAPE = (32, 32, 3)
def load_data():
  (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
  
  x_train = x_train.reshape(50000, 32, 32, 3)
  x_test = x_test.reshape(10000, 32, 32, 3)
  
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

  #regularization

  #x_train모든 값의 평균
  mean = np.mean(x_train, axis=(0,1,2,3))
  std = np.std(x_train, axis=(0,1,2,3))
  x_train = (x_train - mean) / (std + 1e-7)
  x_test = (x_test - mean) / (std + 1e-7)

  y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
  y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

  return (x_train, y_train), (x_test, y_test)

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#이미지 보강
datagen = ImageDataGenerator(
    rotation_range = 30, #30도 회전
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip=True,
)
datagen.fit(x_train)

In [32]:
#심층 cnn
def build_deep_model(input_shape, num_classes):

  model = models.Sequential()
  #첫번째 블록
  model.add(layers.Convolution2D(32, (3,3), padding='same', input_shape=input_shape,activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Convolution2D(32, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Dropout(0.2))

  model.add(layers.Convolution2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Convolution2D(64, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Dropout(0.3))

  model.add(layers.Convolution2D(128, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Convolution2D(128, (3,3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Dropout(0.4))

  model.add(layers.Flatten())
  model.add(layers.Dense(num_classes, activation='softmax'))

  model.summary()
  return model

In [28]:
(x_train, y_train), (x_test, y_test) = load_data()
model = build_deep_model(INPUT_SHAPE, NUM_CLASSES)
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

batch_size = 64
model.fit(x_train,y_train, batch_size=batch_size, epochs=EPOCHS, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print('\nTest Score : ',score[0])
print('\nTest Accuracy : ',score[1])

142/782 [====>.........................] - ETA: 6:12 - loss: 0.4180 - accuracy: 0.8571

KeyboardInterrupt: ignored

In [33]:
#with data generator
batch_size = 64
model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size), epochs=EPOCHS, verbose=1, validation_data=(x_test,y_test))

model_json = model.to_json()
with open('model.json', 'w') as json_file:
  json_file.write(model_json)
model.save_weights('model.h5')

#test
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f ' %(score[1]*100, score[0]))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
782/782 [==============================] - 494s 630ms/step - loss: 0.9799 - accuracy: 0.6656 - val_loss: 0.6286 - val_accuracy: 0.7825
Epoch 2/20
782/782 [==============================] - 488s 624ms/step - loss: 0.8846 - accuracy: 0.6973 - val_loss: 0.6964 - val_accuracy: 0.7671
Epoch 3/20
782/782 [==============================] - 487s 622ms/step - loss: 0.8402 - accuracy: 0.7114 - val_loss: 0.8517 - val_accuracy: 0.7418
Epoch 4/20
782/782 [==============================] - 492s 629ms/step - loss: 0.8095 - accuracy: 0.7211 - val_loss: 0.6497 - val_accuracy: 0.7840
Epoch 5/20
514/782 [==================>...........] - ETA: 2:41 - loss: 0.7896 - accuracy: 0.7287

KeyboardInterrupt: ignored